##### Copyright 2023 Google LLC

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Setup

### Install & import

In [ ]:
!pip install -U -q google-generativeai

In [ ]:
# Install the client library and import necessary modules.
import google.generativeai as genai

import base64
import copy
import hashlib
import io
import json
import mimetypes
import pathlib
import pprint
import requests


import PIL.Image
import IPython.display
from IPython.display import Markdown

### Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

## Set the API key

Add your API_KEY to the secrets manager in the left panel "🔑".

In [ ]:
from google.colab import userdata

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

In [ ]:
# Configure the client library by providing your API key.
genai.configure(api_key=GOOGLE_API_KEY)

### Parse the arguments

In [ ]:
model = "gemini-1.5-flash" # @param {isTemplate: true}
contents_b64 = 'W3sicGFydHMiOiBbeyJ0ZXh0Ijoid2hhdCdzIGluIHRoaXMgcGljdHVyZToifSwgeyJpbWFnZSI6IHsiaW1hZ2VfdXJsIjogImh0dHBzOi8vc3RvcmFnZS5nb29nbGVhcGlzLmNvbS9nZW5lcmF0aXZlYWktZG93bmxvYWRzL2ltYWdlcy9zY29uZXMuanBnIn19XX1d' # @param {isTemplate: true}
generation_config_b64 = "e30=" # @param {isTemplate: true}
safety_settings_b64 = "e30=" # @param {isTemplate: true}

gais_contents = json.loads(base64.b64decode(contents_b64))

generation_config = json.loads(base64.b64decode(generation_config_b64))
safety_settings = json.loads(base64.b64decode(safety_settings_b64))

stream = False

In [ ]:
contents

In [ ]:
generation_config

In [ ]:
safety_settings

### [Optional] Show the conversation

This section displays the conversation received from Google AI Studio.

In [ ]:
def show_file(file_data):
  mime_type=file_data['mime_type']
  if drive_id:=file_data.get('drive_id', None):
    path = next(pathlib.Path(f'/gdrive/.shortcut-targets-by-id/{drive_id}').glob('*'))
    name = path
    data = path.read_bytes()
  elif url:=file_data.get('url', None):
    response = requests.get(url)
    name = url
    data = response.content
  elif data := file_data.get('inline_data'):
    name = None
  else:
    raise ValueError('Either `drive_id`, `url` or `inline_data` must be provided.')

  if mime_type.startswith('image/'):
    image = IPython.display.Image(data=data, format=mimetypes.guess_extension(mime_type).strip('.'), width=256)
    IPython.display.display(image)
    print()
  else:
    print("File:\n    name: {name}\n    mime_type: {mime_type}")


In [ ]:
for content in gais_contents:
  if role:=content.get('role', None):
    print("Role:", role, "\n")

  for n, part in enumerate(content['parts']):
    if text := part.get('text', None):
      print(text, "\n")

    elif file_data:=part.get('file_data', None):
      show_file(file_data)

  print('-'*80, "\n")

### Convert & upload files

In [ ]:
def normalize_file_data(file_data):
  mime_type=file_data['mime_type']
  if drive_id:=file_data.pop('drive_id', None):
    path = next(pathlib.Path(f'/gdrive/.shortcut-targets-by-id/{drive_id}').glob('*'))
    print('Uploading:', str(path))
    file_info = genai.upload_file(path=path, mime_type=mime_type)
    file_data['file_uri'] = file_info.uri
    return

  if url:=file_data.pop('url', None):
    response = requests.get(url)
    data = response.content
    name = url.split('/')[-1]
    hash = hashlib.sha256(data).hexdigest()
    path = tempfiles/hash
    path.write_bytes(data)
    print('Uploading:', url)
    file_info = genai.upload_file(path, display_name=name, mime_type=mime_type)
    file_data['file_uri'] =  file_info.uri
    return

  if 'inline_data' in file_data:
    return


  raise ValueError('Either `drive_id`, `url` or `inline_data` must be provided.')

In [ ]:
tempfiles = pathlib.Path(f"tempfiles")
tempfiles.mkdir(parents=True, exist_ok=True)

contents = copy.deepcopy(gais_contents)

for content in contents:
  for n, part in enumerate(content['parts']):
    if file_data:=part.get('file_data', None):
      normalize_file_data(file_data)


Here is the coverted `Content`:

In [ ]:
contents

### Call the API

In [ ]:
# Call the model and print the response.
gemini = genai.GenerativeModel(model_name=model)

response = gemini.generate_content(
    contents,
    generation_config=generation_config,
    safety_settings=safety_settings,
    stream=False)

In [ ]:
if generation_config.get('candidate_count', 1) == 1:
  display(Markdown(response.text))
else:
  print(response.candidates)

In [ ]:
response.candidates